[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kamalkraj/minGPT-TF/blob/master/play_math.ipynb)

In [ ]:
#This is original 2 digs example

In [1]:
! pip install fastprogress==0.2.3

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1673, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1305, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 249, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1283, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [2]:
import math
import numpy as np
import tensorflow as tf
from mingpt.model import GPT, GPTConfig

In [3]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [4]:
class AdditionDataset():
    """
    Returns addition problems of up to some number of digits in the inputs. Recall
    that all GPT cares about are sequences of integers, and completing them according to
    patterns in the data. Therefore, we have to somehow encode addition problems
    as a sequence of integers.
    
    The sum of two n-digit numbers gives a third up to (n+1)-digit number. So our
    encoding will simply be the n-digit first number, n-digit second number, 
    and (n+1)-digit result, all simply concatenated together. Because each addition
    problem is so structured, there is no need to bother the model with encoding
    +, =, or other tokens. Each possible sequence has the same length, and simply
    contains the raw digits of the addition problem.
    
    As a few examples, the 2-digit problems:
    - 85 + 50 = 135 becomes the sequence [8, 5, 5, 0, 1, 3, 5]
    - 6 + 39 = 45 becomes the sequence [0, 6, 3, 9, 0, 4, 5]
    etc.
    
    We will also only train GPT on the final (n+1)-digits because the first
    two n-digits are always assumed to be given. So when we give GPT an exam later,
    we will e.g. feed it the sequence [0, 6, 3, 9], which encodes that we'd like
    to add 6 + 39, and hope that the model completes the integer sequence with [0, 4, 5]
    in 3 sequential steps.
    
    fun exercise: does it help if the result is asked to be produced in reverse order?
    """

    def __init__(self, ndigit, split):
        self.split = split # train/test
        self.ndigit = ndigit
        self.vocab_size = 10 # 10 possible digits 0..9
        # +1 due to potential carry overflow, but then -1 because very last digit doesn't plug back
        self.block_size = ndigit + ndigit + ndigit + 1 - 1
        
        # split up all addition problems into either training data or test data
        num = (10**self.ndigit)**2 # total number of possible combinations
        r = np.random.RandomState(1337) # make deterministic
        perm = r.permutation(num)
        num_test = min(int(num*0.2), 1000) # 20% of the whole dataset, or only up to 1000
        self.ixes = perm[:num_test] if split == 'test' else perm[num_test:]

    def __len__(self):
        return self.ixes.size

    def __iter__(self):
        # given a problem index idx, first recover the associated a + b
        #Save the train and test data to file
        #fileName = './dataset/train_set_math.txt' if self.split == 'test' else './dataset/test_set_math.txt'
        #file = open(fileName, 'a')
        
        for idx in range(self.__len__()):
            idx = self.ixes[idx]
            nd = 10**self.ndigit
            a = idx // nd
            b = idx %  nd
            c = a + b
            render = f'%0{self.ndigit}d%0{self.ndigit}d%0{self.ndigit+1}d' % (a,b,c) # e.g. 03+25=28 becomes "0325028"    
            #file.write(render+'\n')
            dix = [int(s) for s in render] # convert each character to its token index
            # x will be input to GPT and y will be the associated expected outputs
            x = dix[:-1]
            y = dix[1:] # predict the next token in the sequence
            y[:self.ndigit*2-1] = [-1] * (self.ndigit*2-1) # we will only train in the output locations. -100 will mask loss to zero
            x = tf.convert_to_tensor(x,dtype=tf.int32)
            y = tf.convert_to_tensor(y,dtype=tf.int32)
            yield x, y
            
        #file.close()    
    __call__ = __iter__

In [5]:
# create a dataset for e.g. 2-digit addition
ndigit = 2
train_dataset_gen = AdditionDataset(ndigit=ndigit, split='train')
test_dataset_gen = AdditionDataset(ndigit=ndigit, split='test')

In [6]:
train_dataset = tf.data.Dataset.from_generator(train_dataset_gen,(tf.int32,tf.int32))
test_dataset = tf.data.Dataset.from_generator(test_dataset_gen,(tf.int32,tf.int32))

In [7]:
# initialize a baby GPT model
mconf = GPTConfig(train_dataset_gen.vocab_size, train_dataset_gen.block_size, 
                  n_layer=2, n_head=4, n_embd=128)
# model = GPT(mconf)

In [8]:
from mingpt.trainer import Trainer, TrainerConfig

# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=10, batch_size=1024, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=1024, final_tokens=50*len(train_dataset_gen)*(ndigit+1),
                      num_workers=4)
trainer = Trainer(GPT, mconf, train_dataset, len(train_dataset_gen), test_dataset, len(test_dataset_gen), tconf, device='GPU:1')

config.vocab_size=10,config.n_embd=128
config.block_size=6,config.n_embd=128
config.embd_pdrop=0.1
config.n_embd=128,config.n_head=4,config.attn_pdrop=0.1,config.resid_pdrop=0.1,config.n_layer=2


In [9]:
trainer.train()

epoch 1: train loss 6.07625. lr 5.994512e-04
epoch 1: test loss 5.68791.
config.ckpt_path is not set, the gpt train model weight will not be saved
epoch 2: train loss 5.43125. lr 5.977197e-04
epoch 2: test loss 4.97180.
config.ckpt_path is not set, the gpt train model weight will not be saved
epoch 3: train loss 4.92965. lr 5.948115e-04
epoch 3: test loss 4.72174.
config.ckpt_path is not set, the gpt train model weight will not be saved
epoch 4: train loss 4.68043. lr 5.907379e-04
epoch 4: test loss 4.50538.
config.ckpt_path is not set, the gpt train model weight will not be saved
epoch 5: train loss 4.44617. lr 5.855153e-04
epoch 5: test loss 4.18209.
config.ckpt_path is not set, the gpt train model weight will not be saved
epoch 6: train loss 4.20597. lr 5.791641e-04
epoch 6: test loss 3.96829.
config.ckpt_path is not set, the gpt train model weight will not be saved
epoch 7: train loss 4.03468. lr 5.717095e-04
epoch 7: test loss 3.78269.
config.ckpt_path is not set, the gpt train mo

In [10]:
trainer.display_model()

Model: "gpt"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  1280      
                                                                 
 dropout (Dropout)           multiple                  0         
                                                                 
 encoder_layer (EncoderLayer  multiple                 198272    
 )                                                               
                                                                 
 encoder_layer_1 (EncoderLay  multiple                 198272    
 er)                                                             
                                                                 
 layer_normalization_4 (Laye  multiple                 256       
 rNormalization)                                                 
                                                               

In [11]:
from mingpt.utils import sample

In [12]:
def give_exam(dataset, batch_size=32, max_batches=-1, printResult=False):
    
    results = []
    
    loader = dataset.batch(batch_size)
    for b, (x, y) in enumerate(loader):
        d1d2 = x[:, :ndigit*2]
        d1d2d3 = sample(trainer.model, d1d2, ndigit+1)
        d3 = d1d2d3[:, -(ndigit+1):]
        factors = tf.convert_to_tensor([[10**i for i in range(ndigit+1)][::-1]])
        # decode the integers from individual digits
        d1i = tf.reduce_sum((d1d2[:,:ndigit] * factors[:,1:]),axis=1)
        d2i = tf.reduce_sum((d1d2[:,ndigit:ndigit*2] * factors[:,1:]),axis=1)
        d3i_pred = tf.reduce_sum((d3 * factors),axis=1)
        d3i_gt = d1i + d2i
        correct = (d3i_pred == d3i_gt) # Software 1.0 vs. Software 2.0 fight RIGHT on this line, lol
        for i in range(x.shape[0]):
            results.append(int(correct[i]))
            judge = 'YEP!!!' if correct[i] else 'NOPE'
            if not correct[i]:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)========" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))
            elif printResult:
                print("GPT claims that %03d + %03d = %03d (gt is %03d; %s)" 
                      % (d1i[i], d2i[i], d3i_pred[i], d3i_gt[i], judge))                
        
        if max_batches >= 0 and b+1 >= max_batches:
            break

    print("final score: %d/%d = %.2f%% correct" % (np.sum(results), len(results), 100*np.mean(results)))

In [13]:
give_exam(train_dataset,batch_size=128,max_batches=-1, printResult=False)

GPT claims that 047 + 017 = 079 (gt is 064; NOPE)========
GPT claims that 034 + 055 = 099 (gt is 089; NOPE)========
GPT claims that 018 + 027 = 046 (gt is 045; NOPE)========
GPT claims that 092 + 001 = 099 (gt is 093; NOPE)========
GPT claims that 086 + 028 = 111 (gt is 114; NOPE)========
GPT claims that 086 + 030 = 111 (gt is 116; NOPE)========
GPT claims that 047 + 033 = 089 (gt is 080; NOPE)========
GPT claims that 061 + 080 = 140 (gt is 141; NOPE)========
GPT claims that 086 + 081 = 160 (gt is 167; NOPE)========
GPT claims that 090 + 057 = 140 (gt is 147; NOPE)========
GPT claims that 044 + 072 = 111 (gt is 116; NOPE)========
GPT claims that 066 + 070 = 140 (gt is 136; NOPE)========
GPT claims that 071 + 035 = 109 (gt is 106; NOPE)========
GPT claims that 038 + 090 = 131 (gt is 128; NOPE)========
GPT claims that 022 + 086 = 111 (gt is 108; NOPE)========
GPT claims that 081 + 051 = 131 (gt is 132; NOPE)========
GPT claims that 001 + 066 = 090 (gt is 067; NOPE)========
GPT claims tha

GPT claims that 041 + 045 = 089 (gt is 086; NOPE)========
GPT claims that 006 + 051 = 080 (gt is 057; NOPE)========
GPT claims that 095 + 071 = 150 (gt is 166; NOPE)========
GPT claims that 028 + 099 = 121 (gt is 127; NOPE)========
GPT claims that 088 + 036 = 129 (gt is 124; NOPE)========
GPT claims that 006 + 007 = 023 (gt is 013; NOPE)========
GPT claims that 012 + 051 = 071 (gt is 063; NOPE)========
GPT claims that 024 + 012 = 045 (gt is 036; NOPE)========
GPT claims that 012 + 065 = 091 (gt is 077; NOPE)========
GPT claims that 094 + 009 = 109 (gt is 103; NOPE)========
GPT claims that 061 + 014 = 090 (gt is 075; NOPE)========
GPT claims that 053 + 087 = 141 (gt is 140; NOPE)========
GPT claims that 072 + 067 = 131 (gt is 139; NOPE)========
GPT claims that 082 + 055 = 139 (gt is 137; NOPE)========
GPT claims that 091 + 065 = 150 (gt is 156; NOPE)========
GPT claims that 011 + 065 = 091 (gt is 076; NOPE)========
GPT claims that 058 + 033 = 099 (gt is 091; NOPE)========
GPT claims tha

GPT claims that 001 + 052 = 064 (gt is 053; NOPE)========
GPT claims that 042 + 066 = 112 (gt is 108; NOPE)========
GPT claims that 049 + 051 = 109 (gt is 100; NOPE)========
GPT claims that 064 + 060 = 131 (gt is 124; NOPE)========
GPT claims that 082 + 028 = 111 (gt is 110; NOPE)========
GPT claims that 045 + 011 = 065 (gt is 056; NOPE)========
GPT claims that 080 + 054 = 131 (gt is 134; NOPE)========
GPT claims that 052 + 065 = 111 (gt is 117; NOPE)========
GPT claims that 088 + 073 = 150 (gt is 161; NOPE)========
GPT claims that 043 + 081 = 121 (gt is 124; NOPE)========
GPT claims that 080 + 076 = 150 (gt is 156; NOPE)========
GPT claims that 094 + 062 = 150 (gt is 156; NOPE)========
GPT claims that 021 + 077 = 099 (gt is 098; NOPE)========
GPT claims that 059 + 070 = 131 (gt is 129; NOPE)========
GPT claims that 041 + 059 = 109 (gt is 100; NOPE)========
GPT claims that 094 + 098 = 189 (gt is 192; NOPE)========
GPT claims that 044 + 030 = 089 (gt is 074; NOPE)========
GPT claims tha

GPT claims that 057 + 081 = 141 (gt is 138; NOPE)========
GPT claims that 026 + 092 = 111 (gt is 118; NOPE)========
GPT claims that 018 + 017 = 044 (gt is 035; NOPE)========
GPT claims that 092 + 070 = 160 (gt is 162; NOPE)========
GPT claims that 044 + 095 = 143 (gt is 139; NOPE)========
GPT claims that 048 + 032 = 089 (gt is 080; NOPE)========
GPT claims that 030 + 007 = 034 (gt is 037; NOPE)========
GPT claims that 017 + 088 = 109 (gt is 105; NOPE)========
GPT claims that 055 + 092 = 140 (gt is 147; NOPE)========
GPT claims that 010 + 055 = 075 (gt is 065; NOPE)========
GPT claims that 081 + 028 = 111 (gt is 109; NOPE)========
GPT claims that 040 + 046 = 089 (gt is 086; NOPE)========
GPT claims that 025 + 065 = 099 (gt is 090; NOPE)========
GPT claims that 008 + 083 = 099 (gt is 091; NOPE)========
GPT claims that 062 + 035 = 099 (gt is 097; NOPE)========
GPT claims that 084 + 070 = 150 (gt is 154; NOPE)========
GPT claims that 066 + 046 = 111 (gt is 112; NOPE)========
GPT claims tha

GPT claims that 094 + 018 = 111 (gt is 112; NOPE)========
GPT claims that 085 + 042 = 129 (gt is 127; NOPE)========
GPT claims that 054 + 052 = 109 (gt is 106; NOPE)========
GPT claims that 029 + 085 = 111 (gt is 114; NOPE)========
GPT claims that 077 + 031 = 109 (gt is 108; NOPE)========
GPT claims that 010 + 051 = 066 (gt is 061; NOPE)========
GPT claims that 027 + 054 = 089 (gt is 081; NOPE)========
GPT claims that 053 + 010 = 081 (gt is 063; NOPE)========
GPT claims that 057 + 056 = 112 (gt is 113; NOPE)========
GPT claims that 093 + 045 = 139 (gt is 138; NOPE)========
GPT claims that 045 + 036 = 089 (gt is 081; NOPE)========
GPT claims that 066 + 030 = 099 (gt is 096; NOPE)========
GPT claims that 003 + 025 = 034 (gt is 028; NOPE)========
GPT claims that 044 + 001 = 054 (gt is 045; NOPE)========
GPT claims that 011 + 026 = 045 (gt is 037; NOPE)========
GPT claims that 059 + 065 = 121 (gt is 124; NOPE)========
GPT claims that 027 + 008 = 034 (gt is 035; NOPE)========
GPT claims tha

...

GPT claims that 061 + 055 = 111 (gt is 116; NOPE)========
GPT claims that 096 + 022 = 121 (gt is 118; NOPE)========
GPT claims that 069 + 054 = 121 (gt is 123; NOPE)========
GPT claims that 003 + 066 = 090 (gt is 069; NOPE)========
GPT claims that 025 + 072 = 099 (gt is 097; NOPE)========
GPT claims that 007 + 016 = 034 (gt is 023; NOPE)========
GPT claims that 080 + 088 = 160 (gt is 168; NOPE)========
GPT claims that 048 + 090 = 141 (gt is 138; NOPE)========
GPT claims that 089 + 043 = 139 (gt is 132; NOPE)========
GPT claims that 044 + 053 = 099 (gt is 097; NOPE)========
GPT claims that 081 + 096 = 171 (gt is 177; NOPE)========
GPT claims that 059 + 029 = 099 (gt is 088; NOPE)========
GPT claims that 023 + 053 = 081 (gt is 076; NOPE)========
GPT claims that 007 + 009 = 023 (gt is 016; NOPE)========
GPT claims that 042 + 027 = 079 (gt is 069; NOPE)========
GPT claims that 010 + 006 = 024 (gt is 016; NOPE)========
GPT claims that 074 + 014 = 099 (gt is 088; NOPE)========
GPT claims tha

GPT claims that 029 + 077 = 109 (gt is 106; NOPE)========
GPT claims that 095 + 083 = 160 (gt is 178; NOPE)========
GPT claims that 031 + 065 = 099 (gt is 096; NOPE)========
GPT claims that 003 + 073 = 090 (gt is 076; NOPE)========
GPT claims that 020 + 004 = 034 (gt is 024; NOPE)========
GPT claims that 059 + 087 = 141 (gt is 146; NOPE)========
GPT claims that 081 + 061 = 141 (gt is 142; NOPE)========
GPT claims that 077 + 086 = 160 (gt is 163; NOPE)========
GPT claims that 052 + 029 = 099 (gt is 081; NOPE)========
GPT claims that 062 + 072 = 131 (gt is 134; NOPE)========
GPT claims that 019 + 015 = 035 (gt is 034; NOPE)========
GPT claims that 060 + 038 = 099 (gt is 098; NOPE)========
GPT claims that 052 + 060 = 111 (gt is 112; NOPE)========
GPT claims that 095 + 012 = 112 (gt is 107; NOPE)========
GPT claims that 086 + 098 = 181 (gt is 184; NOPE)========
GPT claims that 013 + 048 = 066 (gt is 061; NOPE)========
GPT claims that 084 + 091 = 170 (gt is 175; NOPE)========
GPT claims tha

GPT claims that 025 + 025 = 059 (gt is 050; NOPE)========
GPT claims that 070 + 006 = 090 (gt is 076; NOPE)========
GPT claims that 063 + 013 = 090 (gt is 076; NOPE)========
GPT claims that 034 + 092 = 121 (gt is 126; NOPE)========
GPT claims that 031 + 023 = 069 (gt is 054; NOPE)========
GPT claims that 027 + 009 = 034 (gt is 036; NOPE)========
GPT claims that 049 + 068 = 121 (gt is 117; NOPE)========
GPT claims that 051 + 001 = 064 (gt is 052; NOPE)========
GPT claims that 037 + 000 = 044 (gt is 037; NOPE)========
GPT claims that 049 + 042 = 099 (gt is 091; NOPE)========
GPT claims that 042 + 092 = 131 (gt is 134; NOPE)========
GPT claims that 048 + 071 = 121 (gt is 119; NOPE)========
GPT claims that 045 + 088 = 131 (gt is 133; NOPE)========
GPT claims that 057 + 044 = 109 (gt is 101; NOPE)========
GPT claims that 037 + 047 = 089 (gt is 084; NOPE)========
GPT claims that 044 + 039 = 089 (gt is 083; NOPE)========
GPT claims that 040 + 083 = 121 (gt is 123; NOPE)========
GPT claims tha

GPT claims that 068 + 087 = 150 (gt is 155; NOPE)========
GPT claims that 001 + 056 = 071 (gt is 057; NOPE)========
GPT claims that 049 + 071 = 121 (gt is 120; NOPE)========
GPT claims that 017 + 056 = 081 (gt is 073; NOPE)========
GPT claims that 061 + 032 = 099 (gt is 093; NOPE)========
GPT claims that 012 + 056 = 079 (gt is 068; NOPE)========
GPT claims that 077 + 078 = 150 (gt is 155; NOPE)========
GPT claims that 056 + 093 = 140 (gt is 149; NOPE)========
GPT claims that 022 + 013 = 045 (gt is 035; NOPE)========
GPT claims that 059 + 014 = 089 (gt is 073; NOPE)========
GPT claims that 070 + 021 = 099 (gt is 091; NOPE)========
GPT claims that 058 + 047 = 109 (gt is 105; NOPE)========
GPT claims that 086 + 056 = 139 (gt is 142; NOPE)========
GPT claims that 044 + 014 = 065 (gt is 058; NOPE)========
GPT claims that 057 + 005 = 072 (gt is 062; NOPE)========
GPT claims that 013 + 004 = 024 (gt is 017; NOPE)========
GPT claims that 085 + 094 = 171 (gt is 179; NOPE)========
GPT claims tha

GPT claims that 087 + 000 = 090 (gt is 087; NOPE)========
GPT claims that 011 + 022 = 045 (gt is 033; NOPE)========
GPT claims that 026 + 065 = 099 (gt is 091; NOPE)========
GPT claims that 031 + 009 = 044 (gt is 040; NOPE)========
GPT claims that 008 + 099 = 109 (gt is 107; NOPE)========
GPT claims that 014 + 091 = 109 (gt is 105; NOPE)========
GPT claims that 094 + 069 = 150 (gt is 163; NOPE)========
GPT claims that 075 + 015 = 099 (gt is 090; NOPE)========
GPT claims that 088 + 013 = 099 (gt is 101; NOPE)========
GPT claims that 016 + 090 = 112 (gt is 106; NOPE)========
GPT claims that 092 + 035 = 129 (gt is 127; NOPE)========
GPT claims that 052 + 077 = 131 (gt is 129; NOPE)========
GPT claims that 013 + 052 = 066 (gt is 065; NOPE)========
GPT claims that 099 + 018 = 111 (gt is 117; NOPE)========
GPT claims that 022 + 070 = 099 (gt is 092; NOPE)========
GPT claims that 084 + 061 = 141 (gt is 145; NOPE)========
GPT claims that 084 + 049 = 139 (gt is 133; NOPE)========
GPT claims tha

In [14]:
give_exam(test_dataset,batch_size=128,max_batches=-1,printResult=True)

GPT claims that 099 + 027 = 129 (gt is 126; NOPE)========
GPT claims that 032 + 045 = 079 (gt is 077; NOPE)========
GPT claims that 023 + 003 = 034 (gt is 026; NOPE)========
GPT claims that 077 + 007 = 090 (gt is 084; NOPE)========
GPT claims that 088 + 020 = 109 (gt is 108; NOPE)========
GPT claims that 062 + 006 = 090 (gt is 068; NOPE)========
GPT claims that 068 + 095 = 160 (gt is 163; NOPE)========
GPT claims that 059 + 024 = 099 (gt is 083; NOPE)========
GPT claims that 064 + 023 = 099 (gt is 087; NOPE)========
GPT claims that 041 + 095 = 131 (gt is 136; NOPE)========
GPT claims that 015 + 089 = 109 (gt is 104; NOPE)========
GPT claims that 047 + 065 = 112 (gt is 112; YEP!!!)
GPT claims that 072 + 050 = 121 (gt is 122; NOPE)========
GPT claims that 046 + 045 = 099 (gt is 091; NOPE)========
GPT claims that 087 + 006 = 099 (gt is 093; NOPE)========
GPT claims that 017 + 061 = 091 (gt is 078; NOPE)========
GPT claims that 003 + 065 = 090 (gt is 068; NOPE)========
GPT claims that 010 

GPT claims that 088 + 040 = 131 (gt is 128; NOPE)========
GPT claims that 054 + 066 = 121 (gt is 120; NOPE)========
GPT claims that 022 + 083 = 109 (gt is 105; NOPE)========
GPT claims that 049 + 075 = 121 (gt is 124; NOPE)========
GPT claims that 066 + 025 = 099 (gt is 091; NOPE)========
GPT claims that 050 + 032 = 089 (gt is 082; NOPE)========
GPT claims that 003 + 017 = 034 (gt is 020; NOPE)========
GPT claims that 051 + 053 = 109 (gt is 104; NOPE)========
GPT claims that 003 + 009 = 023 (gt is 012; NOPE)========
GPT claims that 035 + 054 = 099 (gt is 089; NOPE)========
GPT claims that 090 + 072 = 160 (gt is 162; NOPE)========
GPT claims that 057 + 076 = 131 (gt is 133; NOPE)========
GPT claims that 060 + 024 = 099 (gt is 084; NOPE)========
GPT claims that 099 + 067 = 155 (gt is 166; NOPE)========
GPT claims that 004 + 077 = 091 (gt is 081; NOPE)========
GPT claims that 091 + 002 = 099 (gt is 093; NOPE)========
GPT claims that 057 + 079 = 131 (gt is 136; NOPE)========
GPT claims tha

GPT claims that 010 + 045 = 056 (gt is 055; NOPE)========
GPT claims that 009 + 007 = 023 (gt is 016; NOPE)========
GPT claims that 006 + 052 = 065 (gt is 058; NOPE)========
GPT claims that 060 + 068 = 131 (gt is 128; NOPE)========
GPT claims that 073 + 005 = 090 (gt is 078; NOPE)========
GPT claims that 089 + 021 = 109 (gt is 110; NOPE)========
GPT claims that 018 + 066 = 099 (gt is 084; NOPE)========
GPT claims that 025 + 073 = 099 (gt is 098; NOPE)========
GPT claims that 035 + 008 = 044 (gt is 043; NOPE)========
GPT claims that 065 + 042 = 109 (gt is 107; NOPE)========
GPT claims that 070 + 062 = 131 (gt is 132; NOPE)========
GPT claims that 017 + 018 = 044 (gt is 035; NOPE)========
GPT claims that 073 + 034 = 109 (gt is 107; NOPE)========
GPT claims that 042 + 077 = 121 (gt is 119; NOPE)========
GPT claims that 073 + 059 = 131 (gt is 132; NOPE)========
GPT claims that 092 + 024 = 121 (gt is 116; NOPE)========
GPT claims that 014 + 049 = 066 (gt is 063; NOPE)========
GPT claims tha

GPT claims that 059 + 078 = 131 (gt is 137; NOPE)========
GPT claims that 089 + 014 = 109 (gt is 103; NOPE)========
GPT claims that 027 + 081 = 111 (gt is 108; NOPE)========
GPT claims that 014 + 016 = 034 (gt is 030; NOPE)========
GPT claims that 076 + 081 = 150 (gt is 157; NOPE)========
GPT claims that 069 + 051 = 121 (gt is 120; NOPE)========
GPT claims that 083 + 048 = 131 (gt is 131; YEP!!!)
GPT claims that 086 + 008 = 099 (gt is 094; NOPE)========
GPT claims that 032 + 080 = 111 (gt is 112; NOPE)========
GPT claims that 028 + 058 = 099 (gt is 086; NOPE)========
GPT claims that 096 + 097 = 181 (gt is 193; NOPE)========
GPT claims that 088 + 034 = 129 (gt is 122; NOPE)========
GPT claims that 075 + 091 = 160 (gt is 166; NOPE)========
GPT claims that 056 + 070 = 131 (gt is 126; NOPE)========
GPT claims that 094 + 008 = 109 (gt is 102; NOPE)========
GPT claims that 020 + 020 = 047 (gt is 040; NOPE)========
GPT claims that 079 + 018 = 099 (gt is 097; NOPE)========
GPT claims that 022 

GPT claims that 027 + 029 = 069 (gt is 056; NOPE)========
GPT claims that 096 + 098 = 181 (gt is 194; NOPE)========
GPT claims that 079 + 034 = 111 (gt is 113; NOPE)========
GPT claims that 048 + 057 = 109 (gt is 105; NOPE)========
GPT claims that 027 + 072 = 099 (gt is 099; YEP!!!)
GPT claims that 074 + 091 = 160 (gt is 165; NOPE)========
GPT claims that 011 + 041 = 056 (gt is 052; NOPE)========
GPT claims that 080 + 050 = 131 (gt is 130; NOPE)========
GPT claims that 066 + 097 = 160 (gt is 163; NOPE)========
GPT claims that 036 + 023 = 069 (gt is 059; NOPE)========
GPT claims that 050 + 059 = 109 (gt is 109; YEP!!!)
GPT claims that 068 + 044 = 111 (gt is 112; NOPE)========
GPT claims that 068 + 089 = 150 (gt is 157; NOPE)========
GPT claims that 003 + 018 = 034 (gt is 021; NOPE)========
GPT claims that 024 + 023 = 059 (gt is 047; NOPE)========
GPT claims that 082 + 017 = 109 (gt is 099; NOPE)========
GPT claims that 053 + 099 = 141 (gt is 152; NOPE)========
GPT claims that 062 + 066 

GPT claims that 032 + 070 = 109 (gt is 102; NOPE)========
GPT claims that 038 + 006 = 044 (gt is 044; YEP!!!)
GPT claims that 026 + 001 = 034 (gt is 027; NOPE)========
GPT claims that 069 + 043 = 111 (gt is 112; NOPE)========
GPT claims that 067 + 048 = 111 (gt is 115; NOPE)========
GPT claims that 074 + 016 = 099 (gt is 090; NOPE)========
GPT claims that 065 + 071 = 131 (gt is 136; NOPE)========
GPT claims that 045 + 057 = 099 (gt is 102; NOPE)========
GPT claims that 003 + 064 = 090 (gt is 067; NOPE)========
GPT claims that 076 + 003 = 090 (gt is 079; NOPE)========
GPT claims that 036 + 068 = 109 (gt is 104; NOPE)========
GPT claims that 021 + 045 = 069 (gt is 066; NOPE)========
GPT claims that 084 + 089 = 160 (gt is 173; NOPE)========
GPT claims that 054 + 074 = 131 (gt is 128; NOPE)========
GPT claims that 037 + 023 = 069 (gt is 060; NOPE)========
GPT claims that 041 + 086 = 121 (gt is 127; NOPE)========
GPT claims that 055 + 042 = 099 (gt is 097; NOPE)========
GPT claims that 093 